# 初始化

In [3]:
import os
import time
import threading
from http.server import HTTPServer, SimpleHTTPRequestHandler
from local_packages import *
from dotenv import load_dotenv
from queue import Queue
import concurrent.futures
import random
import json
import openai
from concurrent.futures import ThreadPoolExecutor, as_completed

# 加载环境变量
dotenv_path = os.path.join(os.getcwd(), '.env')
model_path = r'D:\Joining\Models\Text2Vec_base_zh'
# 设置项目根目录和图片目录
project_root = os.path.dirname(dotenv_path)

agentopener=AgentOpener(service_type='qwen')

service=agentopener.service
js=JSProcessor()
divider = LineDivider(r'\？$')
embedder = Embedder(model_path)


class ParseError(Exception):
    def __init__(self, code, message="解析失败"):
        self.code = code
        self.message = message
        super().__init__(self.message)


服务初始化成功


In [4]:
import time
import torch
import numpy as np
import re
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

def clean_key(key):
    cleaned_key = re.sub(r'^\d+\.\d+\s*', '', key).strip()
    return cleaned_key

def clean_value(value):
    lines = value.split('\n')
    cleaned_lines = [line.strip() for line in lines if line.strip()]
    cleaned_value = '\n'.join(cleaned_lines)
    return cleaned_value

# 示例输入文本
text = """1.1 上海交通大学巴黎卓越工程师学院的发展定位是什么？
上海交通大学巴黎卓越工程师学院是上海交通大学的二级学院，依托上海交通大学深厚的工科实力，结合法国精英教育“工程师大学校”的特色，旨在培养具有国际视野、杰出的工程技术人才和企业界领袖。
我们的人才培养理念：
（1）强基多元：注重培养学生在数学与计算机、工程与应用实践等领域的基础科学思维，鼓励知识的交叉融合以适应现代科技发展的新趋势。
（2）全球视野：采用国际化的教学理念和课程体系，实现与全球知名高校的学分互认，让学生在丰富的跨文化交流中增强民族自信心，全面提高他们的全球胜任力。
（3）行业精英：倡导“学习与实践相结合”的教育模式，以工业界的需求为驱动力，提升学生的科学研究能力。学院与行业紧密合作，通过实习、实训等实践环节，强化学生的工程实践能力和创新能力。
1.2 学院的办学环境怎么样？
学院位于上海交通大学主校区（闵行校区），拥有占地面积超1万平的教学和实验室空间。
1.3 与哪所国外高校合作共建？
合作建院的法国工程师学校有巴黎综合理工学校（Ecole Polytechnique）、巴黎国立先进理工学校（ENSTA Paris）、巴黎国立高等电信学校（Telecom Paris）和巴黎国立高等矿业学校（Mines Paris），目前分别隶属于法国规模最大、办学水平最高的两大高校集团，QS世界排名前50位：
巴黎综合理工大学(https://www.polytechnique.edu)
巴黎文理研究大学(http://www.mines-paristech.fr)
在目前已有的作基础上，学院还将与这两所大学展开进一步深入交流与合作。
1.4 学院内学习氛围如何？
学院采取20人小班化教学，师生配比高，确保每一位学生可以得到充足的关注与资源。同时鼓励朋辈间信息交流，共同学习。通过组织优秀的学生代表（学霸团）以及来华交流交换的留学生分享学习心得和宝贵经验，组成学业互助小组，针对课业答疑解惑，为低年级的同学提供了解专业和进一步明确未来发展方向的机会。
1.5 学院男女生比例怎么样？有哪些学生活动？
学院男女比例为3：2。每年组织的新生团队拓展、女生节、一周CP、篮球杯、足球杯、羽毛球赛、法国文化节、团学联破冰节、中秋游园会、迎新年舞会等活动，让学院充满着活泼和浪漫的气息。
链接

1.6 学院培养模式有哪些独特之处？
（1）优秀的生源质量：


（3）20人的小班化教学：大班讲解+小班辅导+特色实验课+特色考核方式(口试课)的多种教学方式保证教学质量。

（4）与领军企业共建课程，校企合作共同指导：法国赛峰集团、施耐德集团、法雷奥集团和ARDIAN(原安盛私募投资公司)等中外知名企业，直接参与制定学院发展规划和培养计划，并与中国北方工业、商飞、阿里云、腾讯、华为、北方工业等开展企业参观考察、实习职位、科研项目、创新竞赛、教学课程、捐赠、奖助学金等深度合作，为学生毕业后就业创造机遇。上海交大和法国合作学校的全球校友网络，助力开启职业生涯。


（6）培养精通科技和人文的复合型人才：30%的社科经管类课程（涉及语言、文学、历史、金融、商业管理、工业设计等领域）。学院开设特色课程“工程师文化课”，定期邀请著名企业、科研机构的专业人员通过商业实战案例分析，分享行业动态，为拓宽学生思维提供广阔的平台。

● IEEE下设四个专业，分流只有1/4的机会。巴黎卓越工程师学院专业任选（面上），保证选到自己的心仪专业。
● 学院专属导师库，覆盖计算机、信息、自动化、网络安全等IEEE专业方向。可灵活地跨方向学科交叉。
● 本科+硕士无缝衔接，直研名额不限制，出国机会100%。有助于避免内卷，让同学沉下心来，扎实提升自己，突破自己。
● 英法双语教学，小班化授课，有助于与世界连结，对接国际组织有天然优势。
● 该专业研究生和本科生80%从事关于人工智能领域的研究，与工业界联系紧密，提供世界知名企业科研和实习机会。

1.7 住宿条件怎么样？
住宿由学校统一安排。可关注“交大家园”微信公众号，了解住宿条件详情。

1. 学院开设有哪些专业？是否可以保研？
开设电子信息类、机械类、能源类三大优势工科专业大类和结合交大文理交叉优势的法语专业（可辅修电子信息类和机械类）。
四个专业均为本硕一贯制培养，保送直研推荐无名额比例限制：吸收法国先进的工程师培养理念和优质教学资源，满足本科阶段学业基本要求，即可获得推荐免试研究生资格。
2. 是否可以根据兴趣自由选择专业？
可以。学院鼓励学生进行多元探索，在充分了解兴趣的前提下做出专业选择。经过基础的通识课及科研训练后，专业选择在大二上学期进行，大三上学期有二次转专业的机会。高考生专业任选，无人数比例限制。
3. 学院的工科专业研究方向包括什么？
3.1 机械类研究领域：
3.1.1 材料科学及相关技术：
   - 核材料：裂变材料、聚变材料等；
   - 材料失效：腐蚀、蠕变、疲劳等；
   - 高分辨表征：中子散射技术、同步辐射高能X射线衍射技术、光谱分析等；
   - 轻金属材料：铝、镁、钛等；
   - 异质材料：复合材料、纳米材料等；
   - 脑机接口：植入式设备、无线通信等；
   - 生物3D打印：组织工程、器官打印等；
   - 多尺度：纳米尺度、微米尺度等；
   - 多物理场材料响应：温度、应力、磁场等；
   - 先进材料与结构：陶瓷、金属合金等；
3.1.2 机械与制造：
   - 微纳机电系统技术：MEMS、微型传感器等；
   - 智能材料结构与柔性器件：形状记忆合金、智能织物等；
   - 表面力学与功能器件：摩擦、磨损等；
   - 航空航天装备动力学：飞行器动力学、空气动力学等；
   - 机构学：连杆、凸轮等；
   - 板材塑性成形理论：冷弯、热弯等；
   - 先进成形技术：增材制造、激光快速成型等；
   - 热挤压工艺：金属成形、粉末冶金等；
   - 数控机床误差检测与补偿：测量仪器、误差补偿方法等；
   - 机电一体化：伺服电机、PLC等；
   - 复杂机电系统设计：系统集成、故障诊断等；
   - 分析与控制：信号处理、系统优化等；
   - 机器人系统故障诊断与智能运维：传感器、算法等；
   - 预测与健康管理：大数据、人工智能等；
   - 统计建模：回归分析、神经网络等；
   - 数据挖掘：关联规则、聚类等；
   - 深度学习：卷积神经网络、递归神经网络等；
3.1.3 机器人与自动化：
   - 机器人技术：工业机器人、服务机器人等；
   - 软体机器人：模糊逻辑、神经网络等；
   - 柔性操作机器人：并联机器人、串联机器人等；
   - 智能制造：物联网、数字双胞等；
   - 自动化装配技术：机器视觉、传感器等；
   - 机器人和智能控制：伺服驱动、控制器等；
   - 数字化制造：CAD/CAM、三维扫描等；
   - 机器人智能加工：五轴机床、数控编程等；
   - 多机器人协同：任务分配、路径规划等；
   - 传感器融合：视觉、红外、超声等；
   - 环境构建：虚拟现实、增强现实等；
3.1.4 电子与信息技术：
   - 电子显微学：透射电子显微镜、扫描电子显微镜等；
   - 晶体塑性力学：晶体缺陷、相变等；
   - 塑性变形尺寸效应：宏观、微观等；
   - 介电：介质材料、电介质物理学等；
   - 压电：压敏材料、电致伸缩材料等；
   - 电卡：非接触IC卡、RFID等；
   - 铁电功能材料与器件：铁电薄膜、氧化物等；
   - 柔性电子：有机电子、薄膜太阳能电池等；
   - 固态制冷：热管理、热交换等；
   - 碳基涂层材料：碳纳米管、石墨烯等；
   - 传感器融合：多传感器信息融合等；
   - 环境构建：人机工程学、可用性设计等；
3.2 能源类研究领域：
3.2.1 清洁能源研究
低碳和零碳燃料设计、制备和应用
可再生合成燃料、电制燃料
空气碳捕集技术
碳资源化利用技术
动力系统先进燃烧技术和能量高效转化
燃料燃烧基础
智能燃料发动机技术
动力系统智能控制和仿真
先进增压及能量利用
电推进及无人驾驶动力系统
混合动力系统
动力系统超净排放控制技术
动力系统排气净化
催化材料设计与开发
先进功能材料

3.2.2 高效能源储存、转换与利用
电池技术
氢燃料电池与电解水制氢
二次电池（锂离子、钠离子、全固态电池等）
液流电池/电化学合成氨、二氧化碳电化学还原
基于计算流体力学（CFD）、分子动力学（MD）与密度泛函理论（DFT）的电池系统宏、微观传质传热数值模拟
太阳能发电及制冷
太阳能高效供热与制冷
太阳能光伏技术及系统应用
分布式能源及储能
热泵与空调新技术
高密度相变储能
能源-水-空气前沿交叉创新（太阳能空气取水、超高储热密度蓄能技术等）
热能与动力工程
热化学利用
生物质、油页岩热化学转化与发电
固体废弃物能源化利用
热力设备自动控制与系统节能
核科学与工程
反应堆热工水力
核安全与系统仿真
反应堆结构材料
核辐射防护与环境保护
核动力关键装备

3.2.3 学科交叉与智慧能源
能源与环保材料制备及应用（与化工材料学科交叉）
纳米微孔和分子有序结构器件自组装热过程研究
核材料及核反应堆结构材料性能及机理研究
数字化能源（与信息工程学科交叉）
先进反应堆物理计算程序开发
人工智能与大数据技术在核工程的应用
先进核反应堆数字孪生系统
能源与能源转化系统评估（与信息工程学科交叉）
基于大数据、物联网和人工智能的能源系统和能源转化系统评价与优化研究
基于大数据和人工智能的能源可持续发展战略研究
碳交易与碳税、碳足迹、水足迹命周期分析
新型电力系统（与电力工程学科交叉）
绿电并网利用
多种能源综合化、热电联产
智能电网、电力智能化

3.3 电子信息类研究领域：
3.3.1 人工智能与机器学习：
   - 人工智能安全
   - 机器学习、深度学习、数据科学
   - 自然语言处理
   - 机器翻译、模式识别、机器思维原理
   - 强化学习、分布式机器学习
3.3.2 计算机视觉与图像处理：
   - 计算机视觉、视频分析、图像理解
   - 视频编码、图像处理、多媒体信号处理
   - 室内外无线定位与导航、3D建模与卫星定位导航
   - 目标识别、导航定位、视觉检测
3.3.3 网络安全与隐私保护：
   - 网络安全、无线安全、物联网安全
   - 信息安全和区块链技术
   - 移动网络安全、隐私增强技术
3.3.4 大数据与数据工程：
   - 大数据分析、大数据存储系统
   - 数据融合、数据挖掘、数据分析
   - 云计算、大数据加速引擎、智能计算加速引擎
3.3.5 信号处理与通信：
   - 无线通信网络、信号/图像处理
   - 语音识别、语音合成、声纹识别
   - 射频收发组件系统集成、卫星定位算法理论
3.3.6 物联网与智能系统：
   - 工业物联网、智能物联网
   - 智能系统、群智感知、无线感知
   - 智能交通系统、无人驾驶车辆
3.3.7 虚拟现实与增强现实：
   - 虚拟现实、增强现实、混合现实技术
   - 智能交互、计算机图形学
3.3.8 控制系统与自动化：
   - 分布式计算与大数据处理
   - 计算机系统性能评测、分析与优化
   - 智能预测控制、网络化系统控制
3.3.9 存储与优化：
   - 数据中心、高性能计算
   - 索引设计、优化算法
3.3.10 其他特定领域：
    - 工业数字孪生
    - 情感计算、自适应学习技术
    - 形式化验证、程序语言理论、知识表示
- 智能人机交互

4. 学院的人才培养有什么独特之处？
宽口径的专业人才培养
以电子信息类、机械类、能源类的专业大类进行培养，专业领域覆盖面广，涵盖主要学科下多个子领域，多方位的学科方向促交叉。
导师组联合培养
留校可跨院系选导师，出国可选合作学校的导师
个性化导师库：全校共享的创新导师矩阵
根据个人研究志趣定制导师团队，导师库面向全校
本硕贯通培养
在自由探索的氛围中低成本试错，培育创新能力
小班化教学
师生比高、与导师充分的交流机会，快速提升外语表达能力

5. 学院的师资力量如何？
为保证教学质量，基础阶段由法国国民教育部选派的一流法籍预科教师与优秀留法博士归国教师搭班授课，教师团队均具备顶尖学府的教育经历（如：巴黎综合理工学校、巴黎高等师范学校、巴黎国立高等矿业学校、巴黎第六大学、巴黎第十一大学、瑞士洛桑联邦理工学院等），并且与工业界有密切合作关系，法籍教师占比50%。工程师阶段70%的课程主要由法国四所主要合作学校选派资深教授来上海交大联合授课，学院还聘请交大电院、机动学院、材料学院、船舶与工程学院的教授和企业研发专家授课，共同指导学生的科学研究和应用实践项目，导师团队副高及以上职称的教师占比超80%。

3.1 学院本硕贯通的学制是如何设置？
学院培养体系按6.5年一贯制进行安排。教学计划分为基础通识阶段+工程师阶段。基础通识阶段注重数理基础，以数学与统计、计算机科学、科学与工程为主，实行跨学科融合教学。结束后自选专业进入工程师阶段学习。
工程师阶段，鼓励学生在学科交叉领域选择研究课题，与领军企业共建课程，校企合作共同指导联合工程实践课题。学生可选择中法导师开展硕士联合指导，跨学院选择交大导师和法国合作院校的导师开展学术交流与合作。
3.2 学院内学习氛围如何？
学院采取20人小班化教学，师生配比高，确保每一位学生可以得到充足的关注与资源。同时鼓励朋辈间信息交流，共同学习。通过组织优秀的学生代表（学霸团）以及来华交流交换的留学生分享学习心得和宝贵经验，组成学业互助小组，针对课业答疑解惑，为低年级的同学提供了解专业和进一步明确未来发展方向的机会。

3.4 学生最后获得的学位是什么？
本科阶段，满足学业基本要求，100%可获得免试推荐研究生资格。完成6.5年整个培养过程，可获得: 上海交通大学学士学位(学历)；上海交通大学硕士学位(学历)；通过法国工程师职衔委员会(CTI)和欧洲工程教育（EUR-ACE）双认证的法国工程师职衔文凭。通过双学位项目，同学还可以额外获得法国合作高校的工程师学位。

4.1 学院有哪些国际交流交换的机会？
所有学生在6.5年学制中均享有有至少六个月的海外交流学习或实习的机会，目前学期交换和双学位交换项目均无需支付额外学费。
学院还设有丰富的寒暑期卓工研学项目（为期1-2周）和双学位项目（为期1年-2.5年）
4.2 国际交流交换是否可以选择欧洲及北美作为目的地？
可以，学院与超十所欧洲、北美、亚洲的顶尖高校合作，提供0.5-2.5年的交流交换项目，如蒙特利尔综合理工、洛桑联邦理工等。
4.3 学生是否可以参加学校的交流项目？
可以，作为交大的一个学院，上海交通大学巴黎卓越工程师学院的学生同其他学院的学生一样，只要符合学校相关交流项目的要求，在与学院教学安排不冲突的前提下，都可以申请参加相关交流项目。
4.4 是不是每个学生都能报名参加双学位项目？
是的，上海交通大学巴黎卓越工程师学院的学生都可以选择申请双学位项目。由法方合作院校根据笔试、面试和平时成绩进行选拔，每届有将近40%的学生进入该项目。
4.5 如何选择学院的国际交流交换项目？
学院为学生的发展提供多种可能性，寒暑期项目可在低年级进行申请，中长期交流交换可根据你的学业目标及生涯规进行个性化安排，最大程度助力实现你的目标。
长期学位交流项交换是指学生赴法方合作院校进行为期18-30个月的研究生联合培养学习，深入了解法国文化和学生生活。符合学业要求的，可额外获得法国合作高校的工程师学位。
选择短期非学位交流交换的同学，可以通过院内、校内众多项目到国外进行一学期的海外学习交流，也可以通过众多行业领军企业与上海交通大学巴黎卓越工程师学院共建课程、联合工程课题等获得海外实习的机会，还可以跟着学院的中法籍导师进入实验室，确定研究方向后申请开展海外科研毕业设计项目。这些海外学习交流经历都会有力地推动学生的成长，为未来的就业或继续深造提供帮助。

5.1 实习、实验室科研等实践教学是如何安排？
贯穿6.5年中分三段渐进式的实习，可推荐国内外知名企业实习岗位/实验室科研，使学生全面深刻地了解企业/实验室科研的工作内容及用人需求，毕业后能够快速适应工作环境。
5.2 实习情况是怎么安排的? 实习期间收费吗？
注重实践与实习是法国工程师培养的主要特色之一，该项目在整个6.5学年里会为学生安排至少三次实习和其他创新实践机会：
-创新创业培训，大一夏季学期，为期4周

-认知实习，大二或大三夏季学期+暑期，为期4~8周；

-研究实习，大三下学期开始，为期14周；

-工程师实习，研究生二、三年级，为期24周。
所有实习期间，不收任何额外费用。并且实习期间，大部分实习公司将支付相应的实习补助。学生可选择到企业/实验室开展实习，学院也可推荐海外实习机会。

6.1 经过学院本硕一贯式培养，硕士毕业生情况如何？
在本科+硕士的特色人才培养模式下，学院毕业生以硕士研究生为主。约四分之三的学生选择就业，兼具工科专业背景和外语沟通能力的复合型人才优势突出，受到中外企业的热烈欢迎，就业行业分布以电子计算机和信息行业、工业制造和新能源行业、金融和咨询管理行业为主，主要就业单位包括北方工业、船舶重工、中国商飞、谷歌、腾讯、华为、英特尔、阿里巴巴、微软、商汤科技、英伟达、字节跳动、泛亚汽车，蔚来，国家电投、南方电网、远景能源、波士顿咨询、埃森哲、普华永道、德勤等。约四分之一的同学选择继续攻读博士学位，前往法国巴黎综合理工大学、瑞士洛桑联邦理工学院、美国普林斯顿大学、宾夕法尼亚大学、德州大学奥斯汀分校，加拿大滑铁卢大学、中国清华大学等世界知名大学深造。
6.2 学院毕业生就业与发展情况如何？
链接就业质量报告
6.3 CTI认证的法国工程师职衔文凭是什么？国际认可度如何？
法国工程师职衔委员会（CTI）类似于美国工程与技术鉴定委员会(ABET)和德国工程教育认证局(ASIN)，法国评估与认证工程师学历教育以及工程师学校资质唯一的主体。通过引入欧盟第三方权威机构周期性地对我院人才培养进行科学的研究与论证，保证了我院工程师教育在国际社会的认可度和人才培养质量的创新性和研究性。
上海交通大学巴黎卓越工程师学院是经过教育部审核批准在上海交大成立的学院，颁发的文凭资格经过中华人民共和国教育部以及法国工程师职衔委员会（CTI）的审核批准和欧洲工程教育（EUR-ACE）认证，在世界范围都得到充分认可。
6.4 法国工程师文凭的含金量如何？
法国工程师文凭，相当于中国的硕士学位，不仅是一张文凭，更是一种社会地位的体现。工程师文凭可以直通博士学习、是跨国企业的敲门砖。法国社会对工程师文凭有一种近乎崇拜的认同感，工程师学院的毕业生有很高的就业率和社会地位。法国百强企业中，60%的总裁和大部分高级管理人员来自于法国工程师学校。
6.5 工程师文凭是由哪个单位颁发的？
上海交通大学巴黎卓越工程师学院已通过法国工程师职衔委员会（CTI）的认证，拥有颁发法国工程师文凭的资格。
6.6拿到的文凭是否同在法国读书拿到的工程师文凭一样？
一样，拿到的工程师职衔文凭与在法国读书拿到的工程师文凭具有同等价值。


7.1 录取时是否一定需要法语基础？
录取时，对法语不作要求，法语零基础亦可。入学后，学院从零基础循序渐进安排法语学习，与法语授课相结合逐渐适应法语教学体系。学院采用以法语为主，中、法、英三语沉浸式的国际化教学氛围，使用原版法语教材，为学生跨文化交流沟通提供良好的环境。毕业时，全体学生最低可达到法语B2水平。
7.2 和交大其它学院招生有什么区别？
招生方面，和交大其它学院专业没有区别，须达到上海交大在该省市的特定批次录取分数线。但是在不同的专业类录取过程中，需要选择填报上海交通大学巴黎卓越工程师学院才会分流到学院。
7.3 我是一名保送生，如何报考贵院？
具备外语类保送生资格的优秀学生，选择报考上海交通大学法语专业（可辅修电子信息类、机械类、能源类专业三个专业之一）可以进入学院。通过上海交通大学的外语类保送生校测进，达到上海交通大学要求的录取分数线，并在专业志愿中填报巴黎卓越工程师学院。
7.3 我是一名高考生，如何报考贵院？
高考选填志愿时，选择电子信息类（巴黎卓越工程师学院，本硕贯通）、机械类（巴黎卓越工程师学院，本硕贯通）、能源类（巴黎卓越工程师学院，本硕贯通）可以进入学院。面向上海市综合评价批次、浙江省三位一体综合评价批次、高考普通批次进行招生。学院作为一个专业分类，纳入上海交通大学统一招生计划，即考生须达到上海交大在该省的各批次录取分数线。报考方式与上海交大其他专业类相同，且需要在专业志愿中填报交大巴黎卓越工程师学院。

进校后二次分流：

每年8月，面向上海交通大学“工科试验班类”专业录取者提供申请分流至学院专业的机会，详情请关注上海交通大学迎新网
7.4 去年在各省的招生录取线是多少？
可在交大本科招生网查询：https://zsb.sjtu.edu.cn/web/jdzsb/3810062.htm
7.5 去年录取线比交大其它学院的高多少？
高考录取线与交大其他工科学院持平。
7.6 是否招收港澳台学生，如何报名？
是，同其它交大学院一样，由交大招生办统一招生录取。具体考试流程及要求请咨询交大本科招生办：34200000。
7.7 是否招收国际留学生，如何报名？
是，直接向学院报名，提供申请材料即可，学院负责遴选。具体可参考学院主页留学生招生相关内容：http://speit.sjtu.edu.cn/indexen!list.action?topNav=165&colorid=9
7.8学费是多少？
2024年，学费将在每人每年45,000元至65,000元的范围内进行调整。（最终的学费金额请以学校官方发布的正式文件为准）
7.9是否有完善的奖助学金体系作为支撑？
学校、学院及其合作企业为品学兼优的同学们提供多渠道的奖助学金，学生受惠面达到70%以上。如：新生学费减免计划含本科学费全额减免、院长奖、ARDIAN企业助学金等。
加双学位交换项目的同学有机会申请中国国家留学基金委奖学金和法国埃菲尔奖学金。

7.10 如何获取最新的招生信息？
咨询电话： 021-34207848, 183 1718 3372 
电子邮箱： admission.speit@sjtu.edu.cn
网站：http://speit.sjtu.edu.cn
"""

result = divider.split_text(text)
cleaned_result = {clean_key(key): clean_value(value) for key, value in result.items()}

# 获取所有的 key_str
key_str_list = list(cleaned_result.keys())

# 嵌入 key_str
model_path = r'D:\Joining\Models\Text2Vec_base_zh'  # 请根据实际情况修改模型路径
embeddings_dict = embedder.embed_list(key_str_list)

# 构建 dict_list
dict_list = []
for key, value in cleaned_result.items():
    dict_list.append({
        "问题": key,
        "答案": value,
        "嵌入向量": embeddings_dict[key].tolist()  # 将 numpy 数组转换为列表，以便可以序列化为 JSON
    })

js.write_json(dict_list,"dict_list.json")


已处理 32/38 个元素，耗时 1.03秒，速度：31.11个元素/秒


In [6]:
question_str = "交大巴院学啥"
dict_list = [
    {"问题": "问题1", "答案": "答案1", "嵌入向量": embedder.get_embedding("问题1")},
    {"问题": "交大巴院学啥", "答案": "答案2", "嵌入向量": embedder.get_embedding("交大巴院学啥")},
    # 其他问题和答案
]

question, answer = embedder.find_most_similar(question_str, dict_list,"嵌入向量","问题","答案")
print(f"最相似的问题: {question}\n答案: {answer}")


最相似的问题: 交大巴院学啥
答案: 答案2
